In [1]:
from simulator.environment import physic_env

In [2]:
import pandas as pd
import moviepy.editor as mpy
import numpy as np
from rpy2.robjects import r, pandas2ri


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
from isaac.visualization import make_frame_curried
from isaac.constants import FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.dataset import read_dataset

In [4]:
from isaac.constants import BASIC_TRAINING_COLS

In [5]:
def transfer(n,x):                                                                                 
    b=[]                                                                                           
    while True:                                                                                    
        s=n//x                                                                                     
        y=n%x                                                                                      
        b=b+[y]                                                                                    
        if s==0:                                                                                   
            break                                                                                  
        n=s                                                                                        
    b.reverse()                                                                                    
    return b

def generate_possible(level, length):                                                            
    possible = []                                                                                  
    for i in range(level**length):                                                                 
        possible.append([0]*(length-len(transfer(i,level)))+transfer(i,level))                     
    return possible

def generate_force(force_possible):                                                             
    force_list = []                                                                                
    for num in force_possible:                                                                     
        num = np.array(num)                                                                       
        force = np.zeros([4,4])                                                                    
        force[1,0] = (num[0])                                                                  
        force[2,:2] = (num[1:3])                                                               
        force[3,:3] = (num[3:])                                                                
        force = force+force.T                                                                      
        force_list.append(force)                                                                   
    return force_list

# Visualising Passive Trials

In [6]:
ALTERNATIVE_TRAINING_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["x", "y", "vx", "vy"]]

In [7]:
rdata_path = "data/experiment1.rdata"
r['load'](rdata_path)

'df.l','df.w','ls.clips','ls.events'


In [8]:
configs = pandas2ri.ri2py_dataframe(r['df.l'])[["trueRelationship", "trueMass", 'lf1', 'lf2', 'lf3', 'lf4', 'lf5', 'lf6']]
is_passive = pandas2ri.ri2py_dataframe(r['df.l']).condition == '0'
idx = []
mass_configs = []
force_configs = []
answers = []

for i, trial_is_passive in enumerate(is_passive):
    if not trial_is_passive:
        continue
    config = configs.iloc[i]
    mass_config = config["trueMass"]
    force_config = config[['lf1', 'lf2', 'lf3', 'lf4', 'lf5', 'lf6']]
    
    trueMass = config.trueMass
    trueForce = config.trueRelationship
    
    idx.append(i)
    answers.append((trueMass, trueForce))
    mass_configs.append(mass_config)
    force_configs.append(force_config.values.astype(int))

In [9]:
all_trials = r['ls.clips']
flat_trials = []
trial_i = 0

for participant in all_trials:
    for trial in participant:
        trial = pandas2ri.ri2py_dataframe(trial)
        
        trial = trial.rename({alt_col: col for col, alt_col in 
                              zip(BASIC_TRAINING_COLS, ALTERNATIVE_TRAINING_COLS)}, axis="columns")
        # print(trial["idControlledObject"].nunique() > 1, is_active[trial_i])
        flat_trials.append(trial)
        trial_i += 1

In [10]:
def transform_mass_answer_to_configuration(answer):
    masses = [1, 1, 1, 1]
    if answer == "same":
        return masses
    if answer == "A":
        masses[0] = 2
        return masses
    masses[1] = 2
    return masses

In [11]:
mass_configs = list(map(transform_mass_answer_to_configuration, mass_configs))

In [12]:
force_configs = generate_force(force_configs)

In [14]:
random_trial

237

In [43]:
random_trial = np.random.choice(np.arange(len(idx)))
c = flat_trials[idx[random_trial]][BASIC_TRAINING_COLS].iloc[0]

m = mass_configs[random_trial]
f = force_configs[random_trial] 

In [44]:
print(c)
print()
print(m)
print()
print(f)
print()
print(answers[random_trial])

o1.x     3.520
o1.y     1.311
o1.vx    9.112
o1.vy   -9.191
o2.x     4.413
o2.y     0.546
o2.vx    4.365
o2.vy    1.630
o3.x     3.960
o3.y     2.027
o3.vx   -7.947
o3.vy   -9.381
o4.x     4.348
o4.y     2.088
o4.vx    9.554
o4.vy   -6.625
Name: 0, dtype: float64

[1, 1, 1, 1]

[[0. 3. 3. 0.]
 [3. 0. 3. 3.]
 [3. 3. 0. 3.]
 [0. 3. 3. 0.]]

('same', 'attract')


In [45]:
cond = {'sls':[{'x':c["o1.x"], 'y':c["o1.y"]},{'x':c["o2.x"], 'y':c["o2.y"]}, {'x':c["o3.x"], 'y':c["o3.y"]}, 
               {'x':c["o4.x"], 'y':c["o4.y"]}],
        'svs':[{'x':c["o1.vx"], 'y':c["o1.vy"]},{'x':c["o2.vx"], 'y':c["o2.vy"]}, {'x':c["o3.vx"], 'y':c["o3.vy"]}, 
               {'x':c["o4.vx"], 'y':c["o4.vy"]}],
        'lf': f,                                   
        'mass': m,                                           
        'timeout': 2700                                                                         
        }

In [46]:
def simulate_trial(trial):                     
    T = 1
    new_env = physic_env([trial], None, None, (3, 2), T, ig_mode=0, prior=None, reward_stop=None)
                                                                                                  
    # mass_answer = get_mass_answer(new_env.cond['mass'])                                            
    # force_answer = get_force_answer(new_env.cond['lf'])                                            
    # print(mass_answer, force_answer)                                                               
                                                                                                   
    for _ in range(int(2700/T)):                                                                
        is_done = new_env.step_passive()                                                           
        if is_done:                                                                                
            break                                                                                  
                                                                                                   
    data = new_env.step_data()                                                                     
    trial_data = pd.DataFrame()                                                                    
                                                                                                   
    for object_id in ["o1", "o2", "o3", "o4"]:                                                     
        for attr in ["x", "y", "vx", "vy"]:                                                        
            trial_data[object_id+"."+attr] = data[object_id][attr]                                 
                                                                                                   
    # trial_data["A"] = mass_answer == "A"                                                           
    # trial_data["B"] = mass_answer == "B"                                                           
    # trial_data["same"] = mass_answer == "same"
                                                                                                   
    # trial_data["attract"] = force_answer == "attract"                                              
    # trial_data["none"] = force_answer == "none"                                                    
    # trial_data["repel"] = force_answer == "repel"                                                  
                                                                                                   
    return trial_data

In [51]:
make_frame = make_frame_curried(flat_trials[idx[random_trial]])
duration = flat_trials[idx[random_trial]].shape[0]/60
clip = mpy.VideoClip(make_frame, duration=duration)
clip.ipython_display(fps=60)
clip.write_videofile("js_simulation.mp4", fps=60)

100%|█████████▉| 2701/2702 [00:24<00:00, 110.25it/s]


[MoviePy] >>>> Building video js_simulation.mp4
[MoviePy] Writing video js_simulation.mp4


100%|█████████▉| 2701/2702 [00:25<00:00, 107.59it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: js_simulation.mp4 



In [48]:
trial_data = simulate_trial(cond)

In [50]:
make_frame = make_frame_curried(trial_data)
duration = trial_data.shape[0]/60
clip = mpy.VideoClip(make_frame, duration=duration)
clip.ipython_display(fps=60)
clip.write_videofile("python_simulation.mp4", fps=60)

100%|█████████▉| 2701/2702 [00:23<00:00, 113.39it/s]


[MoviePy] >>>> Building video python_simulation.mp4
[MoviePy] Writing video python_simulation.mp4


100%|█████████▉| 2701/2702 [00:24<00:00, 110.18it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: python_simulation.mp4 

